In [1]:
# -*- coding: utf-8 -*-

#module importing
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time



# 2. driver경로설정 및 url주소 설정
driverpath = 'C:/Users/Heewon/driver/chromedriver.exe'
dr = webdriver.Chrome(driverpath)

####

#로그인
dr.get('https://www.kita.net/login/login.do?geturl=http://stat.kita.net/stat/pmsd/china/ChinaWholeList.screen')
dr.find_element_by_id('p_member_id').send_keys('kedkorea12')
dr.find_element_by_id('p_passwd').send_keys('ked2363!')
dr.find_element_by_xpath('''//*[@id="myform"]/fieldset/button''').click()

####

df_tot = pd.DataFrame(columns = ['year','month','country','SC','S_code', 'export', 'import', 'page'])

#한국 URL
naraURL = ['http://stat.kita.net/stat/kts/pum/ItemImpExpList.screen']
nara = ['한국']
for site, 나라 in zip(naraURL, nara) :
    #사이트 진입
    dr.get(site)
    #누계 -> 당월
    dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[4]/select/option[1]''').click()
    #100개씩 보기
    dr.find_element_by_xpath('''//*[@id="listCount"]/option[3]''').click()
    
    #연도 43개
    for year_ in [16, 17] : 
        dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[1]/select/option[{0}]'''.format(year_)).click()
        년 = int(dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[1]/select/option[{0}]'''.format(year_)).text)
        
        
        #월 12개
        for month_ in range(1,13) : 
            
            if ((년 == 2019) and (month_ == 2)) :
                break;
            dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[2]/select/option[{0}]'''.format(month_)).click()
            월 = int(dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[2]/select/option[{0}]'''.format(month_)).text)

                
            #조회
            dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[3]/a''').click()                    
            time.sleep(0.5)
            
            abc=0
            temp = 0
            
            while 1:
                
                
                try :
                    if dr.find_element_by_xpath('''//*[@id="mySheet1"]/tbody/tr[3]/td/div/table/tbody/tr/td/div''').text == '조회된 데이터가 없습니다.' :
                        break;
                except :
        
                    try :
                        
                        abc+=1
                        drt = dr.page_source
                        sou = BeautifulSoup(drt,'html.parser')
                        soup = sou.find('tbody')

                        #데이터 적재
                        코드_소 = []
                        소분류 = []
                        수출액 = []
                        #수출증감률 = []
                        수입액 = []
                        #수입증감률 = []
                        #수지 = []
                        
                        
                        #첫페이지 
                        #데이터 적재
                        
                        if abc==1:
                            i=1
#                             print(soup.findAll('td', {'align' : 'Right'})[i*10+5].text)
#                             print(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
#                             print(soup.findAll('td', {'align' : 'Left'})[i*2].text)

                            for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)):
                                코드_소.append(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                                소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2].text)
                                수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10+5].text)
                                #수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10+6].text)
                                수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10+7].text)
                                #수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10+8].text)
                                #수지.append(soup.findAll('td', {'align' : 'Right'})[i*10+9].text)
                                i+=1
                            df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'import':수입액, 'page':1})
                            df_tot = pd.concat([df_tot, df])
                        else: 
                            i=1
#                             print(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
#                             print(soup.findAll('td', {'align' : 'Left'})[i*2-2].text)
#                             print(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                            for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)+1):
                                코드_소.append(soup.findAll('td', {'align' : 'Left'})[i*2-2].text)
                                소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                                수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                                #수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-4].text)
                                수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10-3].text)
                                #수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-2].text)
                                #수지.append(soup.findAll('td', {'align' : 'Right'})[i*10-1].text)
                                i+=1

                            df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'import':수입액, 'page':1})
                            df_tot = pd.concat([df_tot, df])
                        print('1page done')
                        ####################
                        if (len(sou.find('div', class_='boardnavi').find('span').findAll('a'))+1) == 1:
                            break;
                        
                        for tmp in range(1, (len(sou.find('div', class_='boardnavi').find('span').findAll('a'))+1)):
                            tmpt = int(dr.find_element_by_xpath('''//*[@id="pageArea"]/span/a[{0}]'''.format(tmp)).text)
                            print(temp, tmpt)
                            if temp > tmpt :
                                slack.chat.post_message('#analysis', '{0}  error'.format(년*100000 +월*1000+tmpt))
                                break;
                            print('{0}  done'.format(년*100000 +월*1000+tmpt))
                            dr.find_element_by_xpath('''//*[@id="pageArea"]/span/a[{0}]'''.format(tmp)).click()
                            time.sleep(0.5)
                            
                            drt = dr.page_source
                            sou = BeautifulSoup(drt,'html.parser')
                            soup = sou.find('tbody')
                            temp+=1
                            #데이터 적재
                            코드_소 = []
                            소분류 = []
                            수출액 = []
                            #수출증감률 = []
                            수입액 = []
                            #수입증감률 = []
                            #수지 = []

                            #데이터 적재
                            i=1
                            print(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                            for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)+1):
                                코드_소.append(soup.findAll('td', {'align' : 'Left'})[i*2-2].text)
                                소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                                수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                                #수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-4].text)
                                수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10-3].text)
                                #수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-2].text)
                                #수지.append(soup.findAll('td', {'align' : 'Right'})[i*10-1].text)
                                i+=1
                            

                            df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'import':수입액, 'page':tmpt})
                            print(df.shape)
                            
                            if df.shape[0] != 100:
                                drt = dr.page_source
                                sou = BeautifulSoup(drt,'html.parser')
                                soup = sou.find('tbody')
                                
                                #데이터 적재
                                코드_소 = []
                                소분류 = []
                                수출액 = []
                                수출증감률 = []
                                수입액 = []
                                수입증감률 = []
                                수지 = []

                                #데이터 적재
                                i=1
                                print(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                                for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)+1):
                                    코드_소.append(soup.findAll('td', {'align' : 'Left'})[i*2-2].text)
                                    소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                                    수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                                    #수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-4].text)
                                    수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10-3].text)
                                    #수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-2].text)
                                    #수지.append(soup.findAll('td', {'align' : 'Right'})[i*10-1].text)
                                    i+=1


                                df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'import':수입액, 'page':tmpt})
                                print(df.shape)
                            
                            df_tot = pd.concat([df_tot, df])
                        
                        
                        if temp > tmpt :
                             break;
                        print('done')    
                        #화살표 넘기기
                        dr.find_element_by_xpath('''//*[@id="pageArea"]/a[2]''').click()
                        time.sleep(1)
                        

                    except:
                        break;
            
            print('#analysis', '{0}  done'.format(년*100 +월))
            
            df_tot.to_csv('./{0}korea2.csv'.format(년), index=False)

1page done
0 2
200401002  done
20,514
(100, 8)
1 3
200401003  done
9,781
(90, 8)
9,781
(100, 8)
2 4
200401004  done
6,313
(100, 8)
3 5
200401005  done
4,456
(100, 8)
4 6
200401006  done
3,293
(100, 8)
5 7
200401007  done
2,537
(100, 8)
6 8
200401008  done
1,941
(100, 8)
7 9
200401009  done
1,530
(100, 8)
8 10
200401010  done
1,249
(100, 8)
done
1page done
9 12
200401012  done
815
(100, 8)
10 13
200401013  done
699
(100, 8)
11 14
200401014  done
587
(100, 8)
12 15
200401015  done
482
(100, 8)
13 16
200401016  done
394
(100, 8)
14 17
200401017  done
331
(100, 8)
15 18
200401018  done
286
(100, 8)
16 19
200401019  done
240
(100, 8)
17 20
200401020  done
199
(100, 8)
done
1page done
18 22
200401022  done
142
(100, 8)
19 23
200401023  done
117
(100, 8)
20 24
200401024  done
96
(100, 8)
21 25
200401025  done
77
(100, 8)
22 26
200401026  done
63
(100, 8)
23 27
200401027  done
50
(100, 8)
24 28
200401028  done
40
(100, 8)
25 29
200401029  done
31
(100, 8)
26 30
200401030  done
25
(100, 8)
done

(100, 8)
6 17
200405017  done
537
(100, 8)
7 18
200405018  done
452
(100, 8)
8 19
200405019  done
315
(100, 8)
9 20
200405020  done
269
(100, 8)
done
1page done
10 22
200405022  done
196
(30, 8)
196
(100, 8)
11 23
200405023  done
161
(100, 8)
12 24
200405024  done
129
(100, 8)
13 25
200405025  done
106
(100, 8)
14 26
200405026  done
87
(100, 8)
15 27
200405027  done
72
(100, 8)
16 28
200405028  done
56
(100, 8)
17 29
200405029  done
43
(100, 8)
18 30
200405030  done
33
(100, 8)
done
1page done
19 32
200405032  done
20
(100, 8)
20 33
200405033  done
14
(100, 8)
21 34
200405034  done
9
(100, 8)
22 35
200405035  done
6
(100, 8)
23 36
200405036  done
3
(90, 8)
3
(100, 8)
24 37
200405037  done
2
(100, 8)
25 38
200405038  done
0
(100, 8)
26 39
200405039  done
0
(100, 8)
27 40
200405040  done
0
(100, 8)
done
1page done
28 42
200405042  done
0
(100, 8)
29 43
200405043  done
0
(100, 8)
30 44
200405044  done
0
(100, 8)
31 45
200405045  done
0
(100, 8)
32 46
200405046  done
0
(100, 8)
33 47
20040

13,278
(100, 8)
2 4
200409004  done
8,382
(100, 8)
3 5
200409005  done
5,885
(100, 8)
4 6
200409006  done
4,221
(100, 8)
5 7
200409007  done
3,088
(100, 8)
6 8
200409008  done
2,505
(100, 8)
7 9
200409009  done
1,987
(100, 8)
8 10
200409010  done
1,686
(100, 8)
done
1page done
9 12
200409012  done
1,121
(60, 8)
1,121
(100, 8)
10 13
200409013  done
930
(100, 8)
11 14
200409014  done
788
(90, 8)
788
(100, 8)
12 15
200409015  done
668
(30, 8)
668
(100, 8)
13 16
200409016  done
574
(100, 8)
14 17
200409017  done
473
(100, 8)
15 18
200409018  done
396
(100, 8)
16 19
200409019  done
332
(100, 8)
17 20
200409020  done
280
(100, 8)
done
1page done
18 22
200409022  done
190
(100, 8)
19 23
200409023  done
159
(100, 8)
20 24
200409024  done
159
(100, 8)
21 25
200409025  done
106
(100, 8)
22 26
200409026  done
86
(90, 8)
86
(100, 8)
23 27
200409027  done
69
(100, 8)
24 28
200409028  done
57
(100, 8)
25 29
200409029  done
44
(100, 8)
26 30
200409030  done
35
(100, 8)
done
1page done
27 32
200409032

0
(100, 8)
50 53
200412053  done
0
(52, 8)
0
(52, 8)
done
1page done
51 51
200412051  done
0
(100, 8)
52 53
200412053  done
0
(52, 8)
0
(52, 8)
done
1page done
53 51
#analysis 200412  done
1page done
0 2
200301002  done
18,264
(100, 8)
1 3
200301003  done
18,264
(100, 8)
2 4
200301004  done
5,615
(100, 8)
3 5
200301005  done
3,971
(100, 8)
4 6
200301006  done
3,035
(100, 8)
5 7
200301007  done
2,279
(100, 8)
6 8
200301008  done
1,806
(60, 8)
1,806
(100, 8)
7 9
200301009  done
1,427
(100, 8)
8 10
200301010  done
1,151
(100, 8)
done
1page done
9 12
200301012  done
749
(100, 8)
10 13
200301013  done
633
(100, 8)
11 14
200301014  done
525
(100, 8)
12 15
200301015  done
525
(100, 8)
13 16
200301016  done
371
(100, 8)
14 17
200301017  done
304
(100, 8)
15 18
200301018  done
261
(100, 8)
16 19
200301019  done
225
(100, 8)
17 20
200301020  done
196
(100, 8)
done
1page done
18 22
200301022  done
135
(100, 8)
19 23
200301023  done
111
(100, 8)
20 24
200301024  done
93
(100, 8)
21 25
200301025  d

0
(8, 8)
0
(8, 8)
done
1page done
54 51
#analysis 200304  done
1page done
0 2
200305002  done
20,284
(100, 8)
1 3
200305003  done
9,522
(100, 8)
2 4
200305004  done
6,357
(100, 8)
3 5
200305005  done
4,362
(100, 8)
4 6
200305006  done
3,286
(90, 8)
3,286
(100, 8)
5 7
200305007  done
2,496
(100, 8)
6 8
200305008  done
2,023
(60, 8)
2,023
(100, 8)
7 9
200305009  done
1,584
(100, 8)
8 10
200305010  done
1,313
(100, 8)
done
1page done
9 12
200305012  done
869
(100, 8)
10 13
200305013  done
719
(100, 8)
11 14
200305014  done
606
(30, 8)
606
(100, 8)
12 15
200305015  done
508
(100, 8)
13 16
200305016  done
436
(100, 8)
14 17
200305017  done
367
(100, 8)
15 18
200305018  done
300
(100, 8)
16 19
200305019  done
257
(100, 8)
17 20
200305020  done
209
(100, 8)
done
1page done
18 22
200305022  done
147
(100, 8)
19 23
200305023  done
124
(90, 8)
124
(100, 8)
20 24
200305024  done
106
(100, 8)
21 25
200305025  done
87
(100, 8)
22 26
200305026  done
71
(100, 8)
23 27
200305027  done
71
(100, 8)
24 2

0
(100, 8)
46 53
200308053  done
0
(100, 8)
47 54
200308054  done
0
(70, 8)
0
(70, 8)
done
1page done
48 51
200308051  done
0
(100, 8)
49 53
200308053  done
0
(100, 8)
50 54
200308054  done
0
(70, 8)
0
(70, 8)
done
1page done
51 51
200308051  done
0
(100, 8)
52 53
200308053  done
0
(90, 8)
0
(100, 8)
53 54
200308054  done
0
(70, 8)
0
(70, 8)
done
1page done
54 51
#analysis 200308  done
1page done
0 2
200309002  done
22,672
(100, 8)
1 3
200309003  done
10,637
(100, 8)
2 4
200309004  done
6,902
(100, 8)
3 5
200309005  done
4,883
(100, 8)
4 6
200309006  done
3,466
(100, 8)
5 7
200309007  done
2,661
(100, 8)
6 8
200309008  done
2,163
(90, 8)
2,163
(100, 8)
7 9
200309009  done
1,764
(100, 8)
8 10
200309010  done
1,411
(100, 8)
done
1page done
9 12
200309012  done
965
(90, 8)
965
(100, 8)
10 13
200309013  done
796
(100, 8)
11 14
200309014  done
675
(100, 8)
12 15
200309015  done
573
(60, 8)
573
(100, 8)
13 16
200309016  done
483
(90, 8)
483
(100, 8)
14 17
200309017  done
395
(100, 8)
15 18
2

0
(100, 8)
37 43
200312043  done
0
(30, 8)
0
(100, 8)
38 44
200312044  done
0
(100, 8)
39 45
200312045  done
0
(100, 8)
40 46
200312046  done
0
(60, 8)
0
(100, 8)
41 47
200312047  done
0
(100, 8)
42 48
200312048  done
0
(100, 8)
43 49
200312049  done
0
(100, 8)
44 50
200312050  done
0
(100, 8)
done
1page done
45 52
200312052  done
0
(60, 8)
0
(100, 8)
46 53
200312053  done
0
(100, 8)
47 54
200312054  done
0
(85, 8)
0
(85, 8)
done
1page done
48 51
200312051  done
0
(100, 8)
49 53
200312053  done
0
(100, 8)
50 54
200312054  done
0
(85, 8)
0
(85, 8)
done
1page done
51 51
200312051  done
0
(100, 8)
52 53
200312053  done
0
(30, 8)
0
(100, 8)
53 54
200312054  done
0
(85, 8)
0
(85, 8)
done
1page done
54 51
#analysis 200312  done


In [2]:
# -*- coding: utf-8 -*-

#module importing
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time



# 2. driver경로설정 및 url주소 설정
driverpath = 'C:/Users/Heewon/driver/chromedriver.exe'
dr = webdriver.Chrome(driverpath)

####

#로그인
dr.get('https://www.kita.net/login/login.do?geturl=http://stat.kita.net/stat/pmsd/china/ChinaWholeList.screen')
dr.find_element_by_id('p_member_id').send_keys('kedkorea12')
dr.find_element_by_id('p_passwd').send_keys('ked2363!')
dr.find_element_by_xpath('''//*[@id="myform"]/fieldset/button''').click()

####

df_tot = pd.DataFrame(columns = ['year','month','country','SC','S_code', 'export', 'import', 'page'])

#한국 URL
naraURL = ['http://stat.kita.net/stat/kts/pum/ItemImpExpList.screen']
nara = ['한국']
for site, 나라 in zip(naraURL, nara) :
    #사이트 진입
    dr.get(site)
    #누계 -> 당월
    dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[4]/select/option[1]''').click()
    #100개씩 보기
    dr.find_element_by_xpath('''//*[@id="listCount"]/option[3]''').click()
    
    #연도 43개
    for year_ in [18, 19, 20] : 
        dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[1]/select/option[{0}]'''.format(year_)).click()
        년 = int(dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[1]/select/option[{0}]'''.format(year_)).text)
        
        
        #월 12개
        for month_ in range(1,13) : 
            
            if ((년 == 2019) and (month_ == 2)) :
                break;
            dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[2]/select/option[{0}]'''.format(month_)).click()
            월 = int(dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[2]/div[2]/select/option[{0}]'''.format(month_)).text)

                
            #조회
            dr.find_element_by_xpath('''//*[@id="contents"]/div[3]/form/fieldset/div[3]/a''').click()                    
            time.sleep(0.5)
            
            abc=0
            temp = 0
            
            while 1:
                
                
                try :
                    if dr.find_element_by_xpath('''//*[@id="mySheet1"]/tbody/tr[3]/td/div/table/tbody/tr/td/div''').text == '조회된 데이터가 없습니다.' :
                        break;
                except :
        
                    try :
                        
                        abc+=1
                        drt = dr.page_source
                        sou = BeautifulSoup(drt,'html.parser')
                        soup = sou.find('tbody')

                        #데이터 적재
                        코드_소 = []
                        소분류 = []
                        수출액 = []
                        수출증감률 = []
                        수입액 = []
                        수입증감률 = []
                        수지 = []
                        
                        
                        #첫페이지 
                        #데이터 적재
                        
                        if abc==1:
                            i=1
#                             print(soup.findAll('td', {'align' : 'Right'})[i*10+5].text)
#                             print(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
#                             print(soup.findAll('td', {'align' : 'Left'})[i*2].text)

                            for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)):
                                코드_소.append(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                                소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2].text)
                                수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10+5].text)
                                #수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10+6].text)
                                수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10+7].text)
                                #수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10+8].text)
                                #수지.append(soup.findAll('td', {'align' : 'Right'})[i*10+9].text)
                                i+=1
                            df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'import':수입액, 'page':1})
                            df_tot = pd.concat([df_tot, df])
                        else: 
                            i=1
#                             print(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
#                             print(soup.findAll('td', {'align' : 'Left'})[i*2-2].text)
#                             print(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                            for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)+1):
                                코드_소.append(soup.findAll('td', {'align' : 'Left'})[i*2-2].text)
                                소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                                수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                                #수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-4].text)
                                수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10-3].text)
                                #수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-2].text)
                                #수지.append(soup.findAll('td', {'align' : 'Right'})[i*10-1].text)
                                i+=1

                            df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'import':수입액, 'page':1})
                            df_tot = pd.concat([df_tot, df])
                        print('1page done')
                        ####################
                        if (len(sou.find('div', class_='boardnavi').find('span').findAll('a'))+1) == 1:
                            break;
                        
                        for tmp in range(1, (len(sou.find('div', class_='boardnavi').find('span').findAll('a'))+1)):
                            tmpt = int(dr.find_element_by_xpath('''//*[@id="pageArea"]/span/a[{0}]'''.format(tmp)).text)
                            print(temp, tmpt)
                            if temp > tmpt :
                                slack.chat.post_message('#analysis', '{0}  error'.format(년*100000 +월*1000+tmpt))
                                break;
                            print('{0}  done'.format(년*100000 +월*1000+tmpt))
                            dr.find_element_by_xpath('''//*[@id="pageArea"]/span/a[{0}]'''.format(tmp)).click()
                            time.sleep(0.5)
                            
                            drt = dr.page_source
                            sou = BeautifulSoup(drt,'html.parser')
                            soup = sou.find('tbody')
                            temp+=1
                            #데이터 적재
                            코드_소 = []
                            소분류 = []
                            수출액 = []
                            수출증감률 = []
                            수입액 = []
                            수입증감률 = []
                            수지 = []

                            #데이터 적재
                            i=1
                            print(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                            for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)+1):
                                코드_소.append(soup.findAll('td', {'align' : 'Left'})[i*2-2].text)
                                소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                                수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                                #수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-4].text)
                                수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10-3].text)
                                #수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-2].text)
                                #수지.append(soup.findAll('td', {'align' : 'Right'})[i*10-1].text)
                                i+=1
                            

                            df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'import':수입액, 'page':tmpt})
                            print(df.shape)
                            
                            if df.shape[0] != 100:
                                drt = dr.page_source
                                sou = BeautifulSoup(drt,'html.parser')
                                soup = sou.find('tbody')
                                
                                #데이터 적재
                                코드_소 = []
                                소분류 = []
                                수출액 = []
                                수출증감률 = []
                                수입액 = []
                                수입증감률 = []
                                수지 = []

                                #데이터 적재
                                i=1
                                print(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                                for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)+1):
                                    코드_소.append(soup.findAll('td', {'align' : 'Left'})[i*2-2].text)
                                    소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2-1].text)
                                    수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10-5].text)
                                    #수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-4].text)
                                    수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10-3].text)
                                    #수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10-2].text)
                                    #수지.append(soup.findAll('td', {'align' : 'Right'})[i*10-1].text)
                                    i+=1


                                df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'import':수입액, 'page':tmpt})
                                print(df.shape)
                            
                            df_tot = pd.concat([df_tot, df])
                        
                        
                        if temp > tmpt :
                             break;
                        print('done')    
                        #화살표 넘기기
                        dr.find_element_by_xpath('''//*[@id="pageArea"]/a[2]''').click()
                        time.sleep(1)
                        

                    except:
                        break;
            
            print('#analysis', '{0}  done'.format(년*100 +월))
            
            df_tot.to_csv('./{0}korea2.csv'.format(년), index=False)

1page done
0 2
200201002  done
15,421
(100, 8)
1 3
200201003  done
7,729
(100, 8)
2 4
200201004  done
5,166
(100, 8)
3 5
200201005  done
3,584
(100, 8)
4 6
200201006  done
2,713
(100, 8)
5 7
200201007  done
2,025
(100, 8)
6 8
200201008  done
1,544
(100, 8)
7 9
200201009  done
1,258
(100, 8)
8 10
200201010  done
1,033
(100, 8)
done
1page done
9 12
200201012  done
692
(100, 8)
10 13
200201013  done
571
(100, 8)
11 14
200201014  done
458
(100, 8)
12 15
200201015  done
381
(100, 8)
13 16
200201016  done
332
(100, 8)
14 17
200201017  done
273
(100, 8)
15 18
200201018  done
229
(100, 8)
16 19
200201019  done
197
(60, 8)
197
(100, 8)
17 20
200201020  done
169
(100, 8)
done
1page done
18 22
200201022  done
119
(90, 8)
119
(100, 8)
19 23
200201023  done
104
(100, 8)
20 24
200201024  done
86
(100, 8)
21 25
200201025  done
68
(100, 8)
22 26
200201026  done
55
(30, 8)
55
(100, 8)
23 27
200201027  done
45
(30, 8)
45
(100, 8)
24 28
200201028  done
38
(100, 8)
25 29
200201029  done
29
(30, 8)
29
(100

9,275
(30, 8)
9,275
(60, 8)
2 4
200209004  done
6,107
(100, 8)
3 5
200209005  done
6,107
(100, 8)
4 6
200209006  done
6,107
(100, 8)
5 7
200209007  done
6,107
(100, 8)
6 8
200209008  done
6,107
(100, 8)
7 9
200209009  done
6,107
(100, 8)
8 10
200209010  done
6,107
(100, 8)
done
1page done
9 1
#analysis 200209  done
1page done
0 1
200210001  done
6,107
(100, 8)
1 2
200210002  done
6,107
(100, 8)
2 3
200210003  done
6,107
(100, 8)
3 5
200210005  done
6,107
(100, 8)
4 6
200210006  done
6,107
(100, 8)
5 7
200210007  done
6,107
(100, 8)
6 8
200210008  done
6,107
(100, 8)
7 9
200210009  done
6,107
(100, 8)
8 10
200210010  done
6,107
(100, 8)
done
1page done
9 1
#analysis 200210  done
1page done
0 1
200211001  done
6,107
(100, 8)
1 2
200211002  done
6,107
(100, 8)
2 3
200211003  done
6,107
(100, 8)
3 5
200211005  done
6,107
(100, 8)
4 6
200211006  done
6,107
(100, 8)
5 7
200211007  done
6,107
(100, 8)
6 8
200211008  done
6,107
(100, 8)
7 9
200211009  done
6,107
(100, 8)
8 10
200211010  done
6

(100, 8)
3 5
200007005  done
6,107
(100, 8)
4 6
200007006  done
6,107
(100, 8)
5 7
200007007  done
6,107
(100, 8)
6 8
200007008  done
6,107
(100, 8)
7 9
200007009  done
6,107
(100, 8)
8 10
200007010  done
6,107
(100, 8)
done
1page done
9 1
#analysis 200007  done
1page done
0 1
200008001  done
6,107
(100, 8)
1 2
200008002  done
6,107
(100, 8)
2 3
200008003  done
6,107
(100, 8)
3 5
200008005  done
6,107
(100, 8)
4 6
200008006  done
6,107
(100, 8)
5 7
200008007  done
6,107
(100, 8)
6 8
200008008  done
6,107
(100, 8)
7 9
200008009  done
6,107
(100, 8)
8 10
200008010  done
6,107
(100, 8)
done
1page done
9 1
#analysis 200008  done
1page done
0 1
200009001  done
6,107
(100, 8)
1 2
200009002  done
6,107
(100, 8)
2 3
200009003  done
6,107
(100, 8)
3 5
200009005  done
6,107
(100, 8)
4 6
200009006  done
6,107
(100, 8)
5 7
200009007  done
6,107
(100, 8)
6 8
200009008  done
6,107
(100, 8)
7 9
200009009  done
6,107
(100, 8)
8 10
200009010  done
6,107
(100, 8)
done
1page done
9 1
#analysis 200009  do

In [108]:
#module importing

import pandas as pd

korea07 = pd.read_csv('./2007korea3.csv', sep = ',')


In [109]:
korea07.iloc[2000:2100,:]

,year,month,country,SC,S_code,export,import,page
2000,2007,1,한국,업라이트(upright)피아노,920110,240,"2,031",1
2001,2007,1,한국,"깨진유리,유리의그밖의웨이스트(waste)ㆍ스크랩(scrap),유리괴(塊)",700100,239,"1,184",1
2002,2007,1,한국,냉장 또는 냉동기구를 넣을 수 있도록 설계 제작된가구,841891,238,45,1
2003,2007,1,한국,기타,441239,237,262,1
2004,2007,1,한국,침포를제외한방적준비기계의것,844832,237,58,1
2005,2007,1,한국,그밖의직물,521029,236,18,1
2006,2007,1,한국,천연진주나양식진주로만든것,711610,235,258,1
2007,2007,1,한국,"토마토 (신선 ,냉장한것 )",70200,235,0,1
2008,2007,1,한국,벽시계 (밧데리 .축전지 또는 전력 구동식의 것 ),910521,234,152,1
2009,2007,1,한국,232.56데시텍스미만192.31데시텍스이상인것(미터식번수43수초과52수이하),520513,234,"2,727",1


In [110]:
korea06 = pd.read_csv('./2006korea1.csv', sep = ',')

In [123]:
korea06.iloc[400:500,:]

,year,month,country,SC,S_code,export,import,page
400,2006,1,한국,금전등록기,847050,"5,957","1,793",5
401,2006,1,한국,기타,30199,"5,952","11,041",5
402,2006,1,한국,선박 또는 보트의 추지기와 블레이드를 제외한 기계류의 부분품,848590,"5,932","9,659",5
403,2006,1,한국,"폭탄 .수류탄 .어뢰 .지뢰 .미사일 ,기타 총포탄 .발사체",930690,"5,923","67,370",5
404,2006,1,한국,플라스틱제의 식탁용품과 주방용품,392410,"5,882","2,110",5
405,2006,1,한국,인쇄용잉크 (흑색제외 ),321519,"5,842","3,910",5
406,2006,1,한국,인조섬유로만든것,620530,"5,836","2,918",5
407,2006,1,한국,기타,292419,"5,673","1,853",5
408,2006,1,한국,"정제한 동의 박 ,뒷면을 붙이지 않은 것 (두께 0. 15MM이하 )",741011,"5,657","6,220",5
409,2006,1,한국,스테인레스강제의 기타 관과 중공프로파일 (원형횡단면의 것 ),730449,"5,635","5,747",5
